In [2]:
import sys
import time
import torch
from torch.backends import cudnn
from matplotlib import colors
import cv2
import numpy as np
import os
import statistics
from skimage import io, draw
from yolox.data.datasets import COCO_CLASSES
from yolox.utils import fuse_model, get_model_info, postprocess, vis
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from loguru import logger

import yolo_detect as yd



import torchvision


import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler

####### 변경점(트래커 코드 변경) 시작 #######
from tracker.method import tracking
from tracker.data import TrackingInfo
####### 변경점(트래커 코드 변경) 끝 #######
  

#클래스 (차량, 오토바이 등) 분류해주고 박스 원래 사이즈로 돌려놓기     
def visual(output, img_info, cls_conf=0.5):   
    ratio = img_info["ratio"]
    
    #아무런 output이 나오지 않을경우 
    if output is None:
        return None, None, None
    output = output.cpu()
    bboxes = output[:, 0:4]
    # preprocessing: resize
    bboxes /= ratio
    cls = output[:, 6] #현재 이미지에서 잡힌 객체들의 번호 차 = 2번 , 신호등 = 9 번 
    scores = output[:, 4] * output[:, 5] #잡힌 객체들의 신뢰도 수준 
    original_bboxes = []
    original_cls = []
    original_scores = []
    
    #박스 원래 사이즈로 돌려놓기 
    x_min = img_info["x_min"]
    y_min = img_info["y_min"]
    for i in range(len(bboxes)):
        box = bboxes[i]
        cls_id = int(cls[i])
        score = scores[i]
        
        #차, 오토바이, 버스, 트럭에 대해서만 박스랑 스코어 생성해서 트래킹에 보내버리기 
        if cls_id in [2,3,5,7] and cls_conf < score :
            original_cls.append(cls_id)
            original_scores.append(score)
            x0 = int(box[0]) + x_min
            y0 = int(box[1]) + y_min
            x1 = int(box[2]) + x_min
            y1 = int(box[3]) + y_min
            original_bboxes.append([x0,y0,x1,y1])

    #output은 나왔지만 차량에 해당 하는 output이 없을경우 (사람만 포착이 되었을경우)     
    if len(original_bboxes) == 0 :
        return None, None, None
    
    original_bboxes = torch.Tensor(original_bboxes)
    original_cls = torch.Tensor(original_cls)
    original_scores = torch.Tensor(original_scores)

    return original_bboxes, original_cls, original_scores

#트래킹 함수 
def tracker(shape_1,shape_0,bboxes,scores):
    ti = TrackingInfo()
    b_np = np.array(bboxes)
    s_np = np.array(scores).reshape(len(scores),1)
    dets = np.concatenate((b_np,s_np),axis=1)
    
    ## 트래커 업데이트 수행
    ti.size = (shape_1,shape_0)
    res_track,_ = tracking(dets, ti, logger=None)
    

    return res_track


#마스크 생성해주기 
def mask(zone,w,h):  
    mask = np.zeros((h, w), dtype=np.uint8)
    mask.fill(255)

    
    zxs = [z for idx, z in enumerate(zone) if idx % 2 == 0]
    zys = [z for idx, z in enumerate(zone) if idx % 2 != 0]
    zxs = np.array(zxs)
    zys = np.array(zys)
    x, y = draw.polygon(zys, zxs)
    mask[x, y] = 128
    return mask

#주차 알고리즘 실현 
def parking(park,frame,mask,bboxes,scores,num_frame,park_count_frame):
    #주차로 표시될 리스트 만들어 주기 
    parking_id = []
    parking_box = []
    parking_score = []
    res_track = tracker(frame.shape[1],frame.shape[0],bboxes,scores)
    #트래킹 아이디값 하나씩 돌기 
    for s in res_track :
        #트래킹 아이디가 없을경우 안지나감 
        if s.get('id') == None:
            continue
        bbox_xmin = int(s['bbox'][0])
        bbox_ymin = int(s['bbox'][1])
        bbox_xmax = int(s['bbox'][2])
        bbox_ymax = int(s['bbox'][3])

        one = bbox_ymin, bbox_xmin
        two = bbox_ymin, bbox_xmax
        three = bbox_ymax, bbox_xmin
        four = bbox_ymax, bbox_xmax

        bbox_h = bbox_ymax - bbox_ymin
        bbox_w = bbox_xmax - bbox_xmin


        track_id = s['id']

        #주차 구역에 차가 진입 했을 경우부터 딕셔너리에 넣어줌  
        if mask[one] == 255 and mask[two] == 255 and mask[three] == 255 and mask[four] == 255:

            d_key = park.keys() #딕셔너리 키 값만 정의  

            #<1>기존에 없는 새로운 아이디가 추가 될 경우                     
            if track_id not in d_key or bool(park) == False:              
                park[track_id] = [num_frame, num_frame, bbox_h, bbox_w, bbox_xmin,bbox_ymin,0,0]

            #<2>아이디가 있을경우 프레임 넘버만 추가 
            else :
                ##프레임값 업데이트 시켜줘서 이전 프레임이랑 비교해서 예를들어 10프레임이상 주차 영역에 있다면 주차라고 할 수 있도록 업데이트
                park[track_id][1] = num_frame
                h = park[track_id][2]
                w = park[track_id][3]
                ymin = park[track_id][5]
                a = h - round(h * 0.3)
                b = h + round(h * 0.3)
                c = w - round(w * 0.3)
                d = w + round(w * 0.3)
                e = w + round(h * 0.3)
                f = bbox_ymin - 30 < ymin < bbox_ymin + 30

                #앞에 차가 지나가서 박스 모양이 이상한 경우는 업데이트 시키지 않음 (비율값으로 +- 를 세팅했으므로 변경 가능 ) 
                #-> 아이디 변경될 경우 박스 모양으로 아이디 구분을 하려는 이유에서 만든것인데 POC에서는 그 부분은 활용 안할 예정 
                if  (a < bbox_h < b) and (c < bbox_w < d )and f:
                    h = bbox_h
                    w = bbox_w
                    park[track_id][6] = park[track_id][4]
                    park[track_id][4] = bbox_xmin
                    park[track_id][5] = bbox_ymin
                    #좌표값이 +- 2 로 이전프레임의 좌표값과 변경이 없을 경우 1을 추가해주고 이게 셋팅값 이상이 될 경우부터 bbox 그림
                    if park[track_id][4] - 2 < park[track_id][6] < park[track_id][4] + 2 : #센터 좌표값으로 변경 
                        park[track_id][7] += 1 

                #주차구역으로 들어와서 10 프레임 이상부터 & 이전 좌표값에서 움직이 20프레임 이상 없을경우 detect
                if int(park[track_id][1]) - int(park[track_id][0]) > park_count_frame and int(park[track_id][7]) > 20 :
                    parking_id.append(track_id)
                    parking_box.append([bbox_xmin,bbox_ymin,bbox_xmax,bbox_ymax])
                    parking_score.append(s['score'])
    return parking_box,parking_score,parking_id




#비디오 읽고, 전체적인 코드 실행 
def proc(video_path,output_file):
    park = {} #주차 딕셔너리 만들기 
    det = yd.YoloDetector_park() #detector 소환      
    det.load() #모델 load
    cap = cv2.VideoCapture(video_path)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) 
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  
    fps = cap.get(cv2.CAP_PROP_FPS)
    print('fps',fps)
    save_path = output_file
    logger.info(f"video save_path is {save_path}")
    vid_writer = cv2.VideoWriter(
        save_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (int(width), int(height))
    )

    cnt = 0
    if cap.isOpened() :    
        while True:
            cnt+=1
            #원본이미지 = frame 
            ret, frame = cap.read()
            if ret == False :
                break
            if cnt < 0:
                continue
            #프레임수 특정값 이상일떄는 break 할때 
            if cnt > 100:
                break
            #현재 프레임
            num_frame = str(round(cap.get(cv2.CAP_PROP_POS_FRAMES)))
            print('frame : ',num_frame)
            #detect 완료후 결과 받기
            outputs, img_info = det.inference(frame,num_frame,img_zone,cut_rate_x, cut_rate_y)
            #박스 사이즈 원본크기로 복구 
            bboxes, class_id, scores = visual(outputs[0], img_info, det.confthre)
            #박스가 없으면 그냥 이미지 내려 보냄 
            if not bboxes == None :
                #주차알고리즘 들어감 
                parking_box,parking_score,parking_id = parking(park,frame,mask,bboxes,scores,num_frame,park_count_frame) 
                #주차된 차량이 있으면 그것만 결과물로 보내기 
                if parking_box is not None :
                    result_frame = det.vis(frame, parking_box, parking_score,parking_id)
                #주차된 차량이 없으면 그냥 이미지만 내보내기 
                else : 
                    result_frame = frame   
            else : result_frame = frame 

            vid_writer.write(result_frame)
            ch = cv2.waitKey(1)
            if ch == 27 or ch == ord("q") or ch == ord("Q"):
                break


            


#############################################마스크###################################################### 
w, h = 1920, 1080
mask_zone = [0, 0, 680, 0, 680, 560, 1080, 560, 1575, 460, 1575, 0, 1919, 0, 1919, 1079, 0, 1079] 
#속도 느려서 아예 만들어 줘야함 
mask = mask(mask_zone,w,h)
############################################################################################################ 


############################################전체 이미자를 영역######################################################  
img_zone = [620, 0, 620, 590, 1650, 590, 1650, 0]
cut_rate_x = 0.3
cut_rate_y = 0.3

#################################주차영역에서 몇프레임 카운트를 한 후 박스를 그린건지#################################
park_count_frame = int(30)

# input_file = 'park/1.avi'
# output_file = 'output/test2.mp4'

input_list = [
    './input/1.mp4',
#     './input/2.mp4',
#     './input/3.mp4',
#     './input/4.mp4',
#     './input/5.mp4',
#     './input/6.mp4',
#     './input/7.mp4',
#     './input/8.mp4',
#     './input/9.mp4',
#     './input/10.mp4',
#     './input/11.mp4',
#     './input/12.mp4',
]
output_list = [
    './output/1.mp4',
#     './output/2.mp4',
#     './output/3.mp4',
#     './output/4.mp4',
#     './output/5.mp4',
#     './output/6.mp4',
#     './output/7.mp4',
#     './output/8.mp4',
#     './output/9.mp4',
#     './output/10.mp4',
#     './output/11.mp4',
#     './output/12.mp4',
]


for idx, input_path in enumerate(input_list):
    print(f"{input_path} start!!")
    proc(input_list[idx],output_list[idx])
    print(f'####################{input_list[idx]} complete#########################')




./input/1.mp4 start!!


2022-06-15 09:00:36.996 | INFO     | yolo_detect:load:35 - Model Summary: Params: 99.07M, Gflops: 281.93
2022-06-15 09:00:37.561 | INFO     | __main__:proc:193 - video save_path is ./output/1.mp4


fps 30.0
frame :  1
frame :  2
frame :  3
frame :  4
frame :  5
frame :  6
frame :  7
frame :  8
frame :  9
frame :  10
frame :  11
frame :  12
frame :  13
frame :  14
frame :  15
frame :  16
frame :  17
frame :  18
frame :  19
frame :  20
frame :  21
frame :  22
frame :  23
frame :  24
frame :  25
frame :  26
frame :  27
frame :  28
frame :  29
frame :  30
frame :  31
frame :  32
frame :  33
frame :  34
frame :  35
frame :  36
frame :  37
frame :  38
frame :  39
frame :  40
frame :  41
frame :  42
frame :  43
frame :  44
frame :  45
frame :  46
frame :  47
frame :  48
frame :  49
frame :  50
frame :  51
frame :  52
frame :  53
frame :  54
frame :  55
frame :  56
frame :  57
frame :  58
frame :  59
frame :  60
frame :  61
frame :  62
frame :  63
frame :  64
frame :  65
frame :  66
frame :  67
frame :  68
frame :  69
frame :  70
frame :  71
frame :  72
frame :  73
frame :  74
frame :  75
frame :  76
frame :  77
frame :  78
frame :  79
frame :  80
frame :  81
frame :  82
frame :  83
fram